In [ ]:
import os
import glob
import rasterio
import numpy as np
from rasterio.transform import Affine

def tiff_to_asc_converter(input_folder, output_folder=None):
    """
    Converte todos os arquivos TIFF de uma pasta para formato ASCII (.asc)
    
    Parâmetros:
    input_folder (str): Caminho da pasta com os arquivos TIFF
    output_folder (str): Caminho da pasta para salvar os arquivos ASC (opcional)
    """
    
    # Se não especificar pasta de saída, cria uma subpasta 'asc_output'
    if output_folder is None:
        output_folder = os.path.join(input_folder, "asc_output")
    
    # Criar pasta de saída se não existir
    os.makedirs(output_folder, exist_ok=True)
    
    # Encontrar todos os arquivos TIFF na pasta
    tiff_files = glob.glob(os.path.join(input_folder, "*.tif")) + \
                 glob.glob(os.path.join(input_folder, "*.tiff"))
    
    if not tiff_files:
        print(" Nenhum arquivo TIFF encontrado na pasta especificada.")
        return
    
    print(f"🔍 Encontrados {len(tiff_files)} arquivo(s) TIFF para converter.")
    
    converted_count = 0
    error_files = []
    
    for tiff_path in tiff_files:
        try:
            # Nome do arquivo de saída
            base_name = os.path.splitext(os.path.basename(tiff_path))[0]
            if base_name.endswith('.tif'):
                base_name = os.path.splitext(base_name)[0]
            
            asc_path = os.path.join(output_folder, f"{base_name}.asc")
            
            print(f"🔄 Convertendo: {os.path.basename(tiff_path)}")
            
            # Abrir arquivo TIFF
            with rasterio.open(tiff_path) as src:
                # Ler dados
                data = src.read(1)  # Lê a primeira banda
                
                # Obter metadados
                transform = src.transform
                crs = src.crs
                nodata = src.nodata
                
                # Substituir valores nodata por -9999 (padrão ASCII)
                if nodata is not None:
                    data = np.where(data == nodata, -9999, data)
                else:
                    # Se não houver nodata definido, usar -9999 para valores NaN
                    data = np.where(np.isnan(data), -9999, data)
                
                # Obter resolução e bounds
                x_res = transform.a
                y_res = -transform.e  # Resolução Y (positiva)
                x_min = transform.c
                y_max = transform.f
                
                # Número de células
                ncols = src.width
                nrows = src.height
                
                # Escrever arquivo ASCII
                with open(asc_path, 'w') as asc_file:
                    # Escrever cabeçalho
                    asc_file.write(f"ncols         {ncols}\n")
                    asc_file.write(f"nrows         {nrows}\n")
                    asc_file.write(f"xllcorner     {x_min}\n")
                    asc_file.write(f"yllcorner     {y_max - nrows * y_res}\n")
                    asc_file.write(f"cellsize      {x_res}\n")
                    asc_file.write("NODATA_value  -9999\n")
                    
                    # Escrever dados
                    for i in range(nrows):
                        line = ' '.join(f"{val:.6f}" for val in data[i, :])
                        asc_file.write(line + '\n')
            
            print(f"✅ Convertido: {os.path.basename(asc_path)}")
            converted_count += 1
            
        except Exception as e:
            error_msg = f"❌ Erro ao converter {tiff_path}: {str(e)}"
            print(error_msg)
            error_files.append((tiff_path, str(e)))
    
    # Resumo final
    print("\n" + "="*50)
    print("📊 RESUMO DA CONVERSÃO")
    print("="*50)
    print(f"📁 Pasta de entrada: {input_folder}")
    print(f"📁 Pasta de saída: {output_folder}")
    print(f"✅ Arquivos convertidos com sucesso: {converted_count}/{len(tiff_files)}")
    
    if error_files:
        print(f"❌ Arquivos com erro: {len(error_files)}")
        for file_path, error in error_files:
            print(f"   - {os.path.basename(file_path)}: {error}")
    else:
        print("🎉 Todas as conversões foram bem-sucedidas!")

def batch_convert_multiple_folders(folders_list, output_base_folder=None):
    """
    Converte TIFF para ASC em múltiplas pastas
    
    Parâmetros:
    folders_list (list): Lista de caminhos de pastas com arquivos TIFF
    output_base_folder (str): Pasta base para saída (opcional)
    """
    
    for folder in folders_list:
        if not os.path.exists(folder):
            print(f"❌ Pasta não encontrada: {folder}")
            continue
            
        if output_base_folder:
            folder_name = os.path.basename(folder.rstrip('/\\'))
            output_folder = os.path.join(output_base_folder, f"{folder_name}_asc")
        else:
            output_folder = None
            
        print(f"\n{'='*60}")
        print(f"📂 PROCESSANDO PASTA: {folder}")
        print(f"{'='*60}")
        
        tiff_to_asc_converter(folder, output_folder)

def main():
    """
    Função principal com interface interativa
    """
    
    print("🔄 CONVERSOR TIFF PARA ASCII (.asc)")
    print("="*40)
    
    while True:
        print("\nOpções:")
        print("1. Converter uma pasta específica")
        print("2. Converter múltiplas pastas")
        print("3. Sair")
        
        choice = input("\nEscolha uma opção (1-3): ").strip()
        
        if choice == '1':
            input_folder = input("Digite o caminho da pasta com os arquivos TIFF: ").strip()
            
            if not os.path.exists(input_folder):
                print("❌ Pasta não encontrada!")
                continue
                
            output_option = input("Deseja especificar pasta de saída? (s/n): ").strip().lower()
            
            if output_option == 's':
                output_folder = input("Digite o caminho da pasta de saída: ").strip()
            else:
                output_folder = None
                
            tiff_to_asc_converter(input_folder, output_folder)
            
        elif choice == '2':
            folders_input = input("Digite os caminhos das pastas separados por ';': ").strip()
            folders_list = [f.strip() for f in folders_input.split(';') if f.strip()]
            
            output_base = input("Digite a pasta base para saída (opcional): ").strip()
            output_base = output_base if output_base else None
            
            batch_convert_multiple_folders(folders_list, output_base)
            
        elif choice == '3':
            print("👋 Saindo do programa...")
            break
            
        else:
            print("❌ Opção inválida! Tente novamente.")

# Função de uso direto (para salvar e usar depois)
def convert_tiff_folder(input_folder, output_folder=None):
    """
    Função simples para converter uma pasta de TIFF para ASC
    """
    tiff_to_asc_converter(input_folder, output_folder)

if __name__ == "__main__":
    main()